# EXPLORING ALIGNED GENETIC SEQUENCES IN PYTHON USING NUMPY

### Introduction
This is the best tutorial ever written!
<dl>
    <dt>SOFTWARE</dt>
    <dd>Python 3.7.1</dd>
    <dd>Numpy 1.15.4</dd>
    <dd>Pandas 0.25.0</dd>
</dl>

### Methods

In [344]:
from sys import getsizeof
import random
import numpy as np
import pandas as pd

#### Let's create a random [nucleotide](https://knowgenetics.org/nucleotides-and-bases/) sequence that is the average size of a Mycobacterial genome (5Mbp), and then compare the amount of space used when stored as a: **string**, **list**, and **ndarray**

In [343]:
seq_list = random.choices('ATCG', k=5000000)
seq_str = ''.join(seq_list)
seq_array = np.array(seq_list)
records = [{'type': type(seq),
            'length': len(seq),
            'bytes': getsizeof(seq), 
            'perItem': getsizeof(seq)/len(seq)} 
           for seq in (seq_list, seq_str, seq_array)]
pd.DataFrame.from_records(records).sort_values('perItem')

,type,length,bytes,perItem
1,<class 'str'>,5000000,5000049,1.000010
2,<class 'numpy.ndarray'>,5000000,20000096,4.000019
0,<class 'list'>,5000000,40215168,8.043034


#### The string is clearly the most efficient storage of the sequence, followed by the ndarray at 4x and finally the list at 8x. However, since we are interested in using the built in methods of a numpy array, let's figure out why it uses more space and how we can use less.

In [246]:
print(f"Data Type: {seq_array.dtype}")
print(f"Item Size: {seq_array.dtype.itemsize} bytes")


Data Type: <U1
Item Size: 4 bytes


#### The default datatype for each nucleotide is a "1 character unicode" that takes up 4 bytes.  We can declare a specific datatype when initializing our numpy array, and the smallest datatype that numpy supports is **int8** at only 1 byte. So, let's convert the nucleotides into integers using the **ord()** function.

In [329]:
int_array = np.array([ord(char) for char in seq_list], dtype=np.int8)  # Convert chars to int and create ndarray
print(f"Sequence: {int_array}")
print(f"Size    : {getsizeof(int_array)} bytes")


Sequence: [67 71 67 ... 84 67 71]
Size    : 5000096 bytes


#### Now we have an ndarray that is nearly the same size as a string!

In [255]:
positions = random.sample(range(0, 5000001), 100)    # Create a list of random integers between 0 and 5000000
mutants = [                                          # Create list of 4 sequence lists with random mutations
    [ord(char) if i not in positions else ord(random.choice('ATCG')) for (i, char) in enumerate(string_sequence)] 
    for _ in range(4)]

In [256]:
matrix = np.stack(np.array(mutant, dtype=np.int8) for mutant in mutants)

In [335]:
print(f"Shape: {matrix.shape}")
matrix

Shape: (4, 5000000)


array([[65, 67, 67, ..., 84, 67, 67],
       [65, 67, 67, ..., 84, 67, 67],
       [65, 67, 67, ..., 84, 67, 67],
       [65, 67, 67, ..., 84, 67, 67]], dtype=int8)

#### Now we have a matrix of 4 aligned sequences with 5000000 nucleotides each. Let's take a look at position 6 *(remember we start counting at 0)* 

In [337]:
print(matrix[:,5])

[84 84 84 84]


#### It looks like all four sequences have the same nucleotide... 84?  Let's use the **chr()** function to convert the integers back to ASCII characters.

In [339]:
[chr(i) for i in matrix[:,5]]    # Convert all values at position 6 to ASCII

['T', 'T', 'T', 'T']

In [346]:
SNPS = [i for i in range(matrix.shape[1]) if len(np.unique(matrix[:,i])) > 1]

In [347]:
print(f"Positions with variability   : {len(SNPS)}")
print(f"Indicies without variability : {set(positions) - set(SNPS)}")

Positions with variability   : 99
Indicies without variability : {2041911}


#### Of the 100 mutated positions, only 1 was mutated to the same nucleotide in all four sequences.